In [1]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [2]:
# Данный блок в области 1 НЕ выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
# запрещается скрывать предупреждения системы
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
# pip install

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, Normalizer
from sklearn.metrics import precision_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

### Область работы 2 (поиск  модели .... )

In [4]:
# Данный блок(и) НЕ выполняются преподавателем в области 2
# блок(и) предназначены для поиска лучшей модели 
# должен быть понятен и очевиден отбор параметров модели
# оставляйте свои комментарии и разъяснения
# 
# Запрещается размещать данные блоки за пределами обасти 2
# Все блоки данной области должны быть выполнены
#
# ЗАПРЕЩАЕТСЯ ИСПОЛЬЗОВАТЬ ТЕСТОВЫЙ НАБОР
#
# Путь к тренировочному набору
# 
path_train = 'train.csv' # содержит только имя файла, без имен папок !!!
df = pd.read_csv(path_train)
df = df.drop_duplicates()
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,6.4,0.17,0.34,1.5,0.091,42.0,135.0,0.99380,3.25,0.49,9.6,7
1,8.0,0.42,0.36,5.0,0.037,34.0,101.0,0.99200,3.13,0.57,12.3,7
2,6.7,0.27,0.25,8.0,0.053,54.0,202.0,0.99610,3.22,0.43,9.3,5
3,5.2,0.21,0.31,1.7,0.048,17.0,61.0,0.98953,3.24,0.37,12.0,7
4,7.6,0.48,0.37,1.2,0.034,5.0,57.0,0.99256,3.05,0.54,10.4,3


In [5]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1725.000000,1725.000000,1725.000000,1725.000000,1725.000000,1725.000000,1725.000000,1725.000000,1725.000000,1725.000000,1725.000000,1725.000000
mean,6.821797,0.283849,0.332545,5.725652,0.044460,34.615072,135.248116,0.993532,3.201449,0.492394,10.734348,6.004638
std,0.853886,0.106090,0.122279,4.570812,0.021236,16.751535,42.848854,0.002830,0.151395,0.121452,1.272796,1.071667
min,3.900000,0.085000,0.000000,0.600000,0.012000,3.000000,10.000000,0.987110,2.830000,0.220000,8.400000,3.000000
25%,6.300000,0.210000,0.270000,1.600000,0.034000,23.000000,105.000000,0.991320,3.100000,0.400000,9.600000,5.000000
50%,6.800000,0.270000,0.320000,4.600000,0.041000,33.000000,130.000000,0.993200,3.190000,0.480000,10.600000,6.000000
75%,7.300000,0.330000,0.380000,8.700000,0.049000,45.000000,164.000000,0.995500,3.300000,0.560000,11.700000,7.000000
max,11.800000,1.100000,1.660000,20.700000,0.290000,124.000000,366.500000,1.001820,3.820000,1.080000,14.200000,9.000000


# Удалил 3 столбца: citric acid, density, sulphates
1) density (плотность), у всех вин примерно одинаковая плотность (997 кг/м3), это и видно в описании, 
очень низкое стандартное отклонение, показатель ни на что особо не влияет 
2) sulphates (сульфаты), в производстве вина сульфаты не используются, используются сульфиты (вот пруф: https://simplewine.ru/articles/q-a/vopros-otvet-5-sulfity-i-drozhzhi-zachem-ikh-dobavlyayut-v-vino-i-vredno-li-eto/#:%7E:text=%D0%A2%D1%83%D1%82%20%D1%81%D1%82%D0%BE%D0%B8%D1%82%20%D1%83%D1%82%D0%BE%D1%87%D0%BD%D0%B8%D1%82%D1%8C%2C%20%D1%87%D1%82%D0%BE%20%D0%B2%2C%D0%BD%D0%B5%20%D0%B8%D1%81%D0%BF%D0%BE%D0%BB%D1%8C%D0%B7%D1%83%D1%8E%D1%82%D1%81%D1%8F%20%D0%BF%D1%80%D0%B8%20%D0%BF%D1%80%D0%BE%D0%B8%D0%B7%D0%B2%D0%BE%D0%B4%D1%81%D1%82%D0%B2%D0%B5%20%D0%B2%D0%B8%D0%BD), даже если предположить, что имелись в виду сульфиты, даже в десертных винах показатель равняется до 0.3, в обычных 0.15, а средний показатель в выборке 0.49, в общем ореинтироваться на это как минимум странно 
3) citric acid (лимонная кислота), в показателе fixed acidity (она же активная кислота) учитывается общий вес всех кислот (лимонная, яблочная, молочная, винная и тд), то есть лимонную кислоту мы учитываем дважды, это путает модель 
(вот пруф: https://cyberleninka.ru/article/n/analiz-otsenki-kachestva-vina-na-osnove-dannyh-o-ego-himicheskom-sostave#:%7E:text=%D0%A4%D0%B8%D0%BA%D1%81%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%BD%D0%B0%D1%8F%20%D0%BA%D0%B8%D1%81%D0%BB%D0%BE%D1%82%D0%BD%D0%BE%D1%81%D1%82%D1%8C%20%D0%BE%D0%B1%D1%83%D1%81%D0%BB%D0%BE%D0%B2%D0%BB%D0%B5%D0%BD%D0%B0%20%D0%BD%D0%B0%D0%BB%D0%B8%D1%87%D0%B8%D0%B5%D0%BC%20%D0%BD%D0%B5%D0%BB%D0%B5%D1%82%D1%83%D1%87%D0%B8%D1%85%2C%D0%BD%D0%B0%D0%BF%D1%80%D0%B8%D0%BC%D0%B5%D1%80%2C%20%D1%82%D0%B0%D0%BA%D0%B8%D1%85%20%D0%BA%D0%B0%D0%BA%20%D1%83%D0%BA%D1%81%D1%83%D1%81%D0%BD%D0%B0%D1%8F%20%D0%BA%D0%B8%D1%81%D0%BB%D0%BE%D1%82%D0%B0)
4) разделил free sulfur dioxide и total sulfur dioxide на 1000, судя по всему, показатели указаны в мг/л, но обычно (да и все остальные показатели кроме pH и alcohol *они особенные*) все указывается в г/л

In [6]:
df.drop(['citric acid', 'density', 'sulphates'], axis = 1, inplace = True)
df['free sulfur dioxide'] = df['free sulfur dioxide'].apply(lambda x : x/1000)
df['total sulfur dioxide'] = df['total sulfur dioxide'].apply(lambda x : x/1000)
df['quality'] = np.where(df['quality'] >= 7, 1, 0)

In [7]:
df.head()

,fixed acidity,volatile acidity,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,pH,alcohol,quality
0,6.4,0.17,1.5,0.091,0.042,0.135,3.25,9.6,1
1,8.0,0.42,5.0,0.037,0.034,0.101,3.13,12.3,1
2,6.7,0.27,8.0,0.053,0.054,0.202,3.22,9.3,0
3,5.2,0.21,1.7,0.048,0.017,0.061,3.24,12.0,1
4,7.6,0.48,1.2,0.034,0.005,0.057,3.05,10.4,0


In [8]:
X, y = df.drop(['quality'], axis = 1), df['quality']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state = 42)

# Силу регуляризации определял методом подбора, заметил, что чем ближе к 0.012, тем лучше))

In [9]:
pipe = Pipeline([('preprocessing', MinMaxScaler()), 
                 ('clf',           LogisticRegression(max_iter = 500))])

scaling = [ MinMaxScaler(), StandardScaler(), RobustScaler(), Normalizer()]

param_grid =[
    {'preprocessing': scaling,'clf__penalty': ['l2'], 
         'clf__solver': ['newton-cg' ,'lbfgs', 'liblinear', 'sag', 'saga'],
    'clf__C' : [0.0119, 0.01189, 0.01188, 0.01187, 0.01186, 0.01185]}, #[1, 10**-1, 10**-2, 10**-3, 10**-4, 10**-5]
    {'preprocessing': scaling,'clf__penalty': ['l1'], 
         'clf__solver': ['liblinear', 'saga'],
    'clf__C' : [0.0119, 0.01189, 0.01188, 0.01187, 0.01186, 0.01185]}, #[0.01, 0.005, 0.015, 0.007, 0.009, 0.013]
    {'preprocessing': scaling,'clf__penalty': ['none'], 
         'clf__solver': ['lbfgs','newton-cg', 'sag', 'saga'],
    'clf__C' : [0.0119, 0.01189, 0.01188, 0.01187, 0.01186, 0.01185]}, #[0.015, 0.016, 0.017, 0.018, 0.019, 0.02, 0.014, 0.013, 0.012]
     {'preprocessing': scaling,'clf__penalty': ['elasticnet'],
      'clf__solver': ['saga'],
    'clf__C' : [0.0119, 0.01189, 0.01188, 0.01187, 0.01186, 0.01185]} #[0.0119, 0.01191, 0.01193, 0.012, 0.012, 0.0121, 0.0122, 0.0123, 0.0124, 0.0125]
    ]

grid = GridSearchCV(pipe, param_grid, cv = 5, scoring = 'precision', return_train_score = True, refit = 'precision')
grid.fit(X_train, y_train)
grid   

C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is i

C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is i

C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is i

C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is i

C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is i

...

C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='non

C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was re

C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='non

C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='non

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing', MinMaxScaler()),
                                       ('clf',
                                        LogisticRegression(max_iter=500))]),
             param_grid=[{'clf__C': [0.0119, 0.01189, 0.01188, 0.01187, 0.01186,
                                     0.01185],
                          'clf__penalty': ['l2'],
                          'clf__solver': ['newton-cg', 'lbfgs', 'liblinear',
                                          'sag', 'saga'],
                          'preprocessing': [MinMaxScaler(), StandardScaler(),
                                            RobustScaler(), Normalizer()...
                          'clf__solver': ['lbfgs', 'newton-cg', 'sag', 'saga'],
                          'preprocessing': [MinMaxScaler(), StandardScaler(),
                                            RobustScaler(), Normalizer()]},
                         {'clf__C': [0.0119, 0.01189, 0.01188, 0.01187, 0.01186,
 

In [10]:
grid_result = pd.DataFrame(grid.cv_results_).sort_values(["rank_test_score",'std_test_score']).T
grid_result

,126,134,142,150,158,166,125,133,141,149,...,271,270,269,268,267,265,264,286,274,287
mean_fit_time,0.005806,0.005647,0.005794,0.006002,0.005911,0.006399,0.004021,0.003799,0.004079,0.004201,...,0.002129,0.002994,0.002001,0.002199,0.001793,0.001999,0.002,0.002998,0.002998,0.001602
std_fit_time,0.000739,0.000436,0.000397,0.00063,0.000182,0.000486,0.000052,0.000748,0.000652,0.000396,...,0.00026,0.000009,0.000006,0.000401,0.000396,0.000012,0.000007,0.000007,0.000006,0.000488
mean_score_time,0.001402,0.001802,0.001204,0.0012,0.001603,0.001397,0.00132,0.001597,0.001316,0.001397,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std_score_time,0.00049,0.000402,0.000405,0.000394,0.000486,0.000489,0.000409,0.000489,0.000406,0.000491,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
param_clf__C,0.0119,0.01189,0.01188,0.01187,0.01186,0.01185,0.0119,0.01189,0.01188,0.01187,...,0.01189,0.01189,0.01189,0.01189,0.0119,0.0119,0.0119,0.01185,0.01188,0.01185
param_clf__penalty,l1,l1,l1,l1,l1,l1,l1,l1,l1,l1,...,elasticnet,elasticnet,elasticnet,elasticnet,elasticnet,elasticnet,elasticnet,elasticnet,elasticnet,elasticnet
param_clf__solver,saga,saga,saga,saga,saga,saga,saga,saga,saga,saga,...,saga,saga,saga,saga,saga,saga,saga,saga,saga,saga
param_preprocessing,RobustScaler(),RobustScaler(),RobustScaler(),RobustScaler(),RobustScaler(),RobustScaler(),StandardScaler(),StandardScaler(),StandardScaler(),StandardScaler(),...,Normalizer(),RobustScaler(),StandardScaler(),MinMaxScaler(),Normalizer(),StandardScaler(),MinMaxScaler(),RobustScaler(),RobustScaler(),Normalizer()
params,"{'clf__C': 0.0119, 'clf__penalty': 'l1', 'clf_...","{'clf__C': 0.01189, 'clf__penalty': 'l1', 'clf...","{'clf__C': 0.01188, 'clf__penalty': 'l1', 'clf...","{'clf__C': 0.01187, 'clf__penalty': 'l1', 'clf...","{'clf__C': 0.01186, 'clf__penalty': 'l1', 'clf...","{'clf__C': 0.01185, 'clf__penalty': 'l1', 'clf...","{'clf__C': 0.0119, 'clf__penalty': 'l1', 'clf_...","{'clf__C': 0.01189, 'clf__penalty': 'l1', 'clf...","{'clf__C': 0.01188, 'clf__penalty': 'l1', 'clf...","{'clf__C': 0.01187, 'clf__penalty': 'l1', 'clf...",...,"{'clf__C': 0.01189, 'clf__penalty': 'elasticne...","{'clf__C': 0.01189, 'clf__penalty': 'elasticne...","{'clf__C': 0.01189, 'clf__penalty': 'elasticne...","{'clf__C': 0.01189, 'clf__penalty': 'elasticne...","{'clf__C': 0.0119, 'clf__penalty': 'elasticnet...","{'clf__C': 0.0119, 'clf__penalty': 'elasticnet...","{'clf__C': 0.0119, 'clf__penalty': 'elasticnet...","{'clf__C': 0.01185, 'clf__penalty': 'elasticne...","{'clf__C': 0.01188, 'clf__penalty': 'elasticne...","{'clf__C': 0.01185, 'clf__penalty': 'elasticne..."
split0_test_score,0.695652,0.695652,0.695652,0.695652,0.695652,0.695652,0.66,0.66,0.66,0.66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# использовал precision
"Данная метрика покажет насколько точно мы предсказываем класс 1, есть ли у нас мусор в предсказанном классе 1 из класса 0. Ошибки в самом классе 1 нас не интересуют". А именно эту задачу мы и решаем. Отбираем вина с качеством 7 и выше (то есть класс 1)

In [11]:
best_params = grid.best_params_
best_precision = grid.best_score_

y_pred = grid.predict(X_test)
precision = precision_score(y_test, y_pred)

In [12]:
best_params

{'clf__C': 0.0119,
 'clf__penalty': 'l1',
 'clf__solver': 'saga',
 'preprocessing': RobustScaler()}

In [13]:
best_precision

0.8359717577108882

In [14]:
model_1 = Pipeline([
    ('preprocessing', RobustScaler()),
    ('clf', LogisticRegression(C = 0.0119, penalty = 'l1', solver = 'saga', max_iter = 100))
])
model_1.fit(X_train, y_train)
y_pred = model_1.predict(X_test)
precision_score(y_test, y_pred)

0.8142857142857143

In [15]:
matrix=confusion_matrix(y_test, y_pred)
ConfusionMatrixDisplay(matrix).plot();